In [22]:
from pathlib import Path
import pandas as pd
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

### Get data

In [12]:
#might have to update path dependning on where you save the files
#uses the track_features_updated located on github
data_folder = Path("track_features_updated/track_features_updated/").glob('**/*')
files_to_read = [x for x in data_folder if x.is_file()]

In [13]:
%%time

df_from_each_file = [pd.read_csv(file) for file in files_to_read]
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    

Wall time: 833 ms


In [14]:
concatenated_df.head()

,track_name,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genre
0,Pure Verleiding,Arjon Oostrom,0.825,0.834,6,-7.363,1,0.1650,0.144,0.0000,0.0786,0.904,135.037,203799,0
1,Beesten Van De Nacht (Official Sensation Waail...,Jeroen Van Zelst,0.685,0.845,5,-7.719,1,0.0556,0.487,0.0175,0.0632,0.651,154.953,145611,0
2,De Woonboot,Stef Ekkel,0.619,0.594,7,-7.626,1,0.0362,0.248,0.0000,0.3930,0.451,97.418,267093,2
3,Daar Mot Een Piemel In,Johnny Gold,0.872,0.910,5,-3.216,1,0.0718,0.443,0.0000,0.0721,0.966,123.956,155493,1
4,Meisjes Van De Nacht,Rob Ronalds,0.722,0.935,8,-4.975,0,0.0450,0.453,0.0000,0.0607,0.842,125.026,182307,1


In [15]:
Droplist = ["track_name","artist_name","genre"]

X = concatenated_df.drop(Droplist,axis = 1)
y = concatenated_df["genre"]

## Multinomial Logistic Regression Model


https://www.datasklr.com/logistic-regression/multinomial-logistic-regression

In [16]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(249543, 12)
(62386, 12)
(249543,)
(62386,)


In [17]:
%%time

#Took 2 minutes

model1 = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg').fit(X_train, y_train)
preds = model1.predict(X_test)

D:\AnacondaPython\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
D:\AnacondaPython\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


Wall time: 2min 2s


D:\AnacondaPython\lib\site-packages\sklearn\utils\optimize.py:211: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "


In [18]:
#Create a confusion matrix
#y_test as first argument and the preds as second argument 
confusion_matrix(y_test, preds)


array([[18774,     0,  1825,   411,     0,     0],
       [ 2280,    91,  6128,    77,     0,     0],
       [ 4495,     1, 16887,   524,     0,     1],
       [   67,     0,  1275,  2935,     0,     1],
       [ 2168,    24,  1963,   170,     0,     0],
       [  713,     0,  1125,   449,     0,     2]], dtype=int64)

In [19]:
print('Accuracy Score:', metrics.accuracy_score(y_test, preds))  

Accuracy Score: 0.6201551630173436


In [21]:
class_report=classification_report(y_test, preds)
print(class_report)

              precision    recall  f1-score   support

           0       0.66      0.89      0.76     21010
           1       0.78      0.01      0.02      8576
           2       0.58      0.77      0.66     21908
           3       0.64      0.69      0.66      4278
           4       0.00      0.00      0.00      4325
           5       0.50      0.00      0.00      2289

    accuracy                           0.62     62386
   macro avg       0.53      0.39      0.35     62386
weighted avg       0.60      0.62      0.54     62386



## Normalized Data

In [23]:
norm_X = preprocessing.scale(X)

In [26]:
norm_X_train, norm_X_test, norm_y_train, norm_y_test = sklearn.model_selection.train_test_split(norm_X, y, test_size = 0.20, random_state = 5)

In [29]:
%%time

#Took 10 minutes

model2 = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg').fit(norm_X_train, norm_y_train)
norm_preds = model2.predict(norm_X_test)

Wall time: 9min 53s


D:\AnacondaPython\lib\site-packages\sklearn\utils\optimize.py:211: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn("newton-cg failed to converge. Increase the "


In [30]:
#Create a confusion matrix
#y_test as first argument and the preds as second argument 
confusion_matrix(norm_y_test, norm_preds)


array([[20582,   196,    79,   129,     1,    23],
       [   93,  8148,   274,    61,     0,     0],
       [  233,    40, 21500,   116,     0,    19],
       [   46,   156,   183,  3887,     0,     6],
       [    0,     1,     0,     0,  4322,     2],
       [   20,     0,    18,     8,     0,  2243]], dtype=int64)

In [31]:
print('Accuracy Score:', metrics.accuracy_score(norm_y_test, norm_preds))  

Accuracy Score: 0.972686179591575


In [32]:
class_report=classification_report(y_test, preds)
print(class_report)

              precision    recall  f1-score   support

           0       0.66      0.89      0.76     21010
           1       0.78      0.01      0.02      8576
           2       0.58      0.77      0.66     21908
           3       0.64      0.69      0.66      4278
           4       0.00      0.00      0.00      4325
           5       0.50      0.00      0.00      2289

    accuracy                           0.62     62386
   macro avg       0.53      0.39      0.35     62386
weighted avg       0.60      0.62      0.54     62386



D:\AnacondaPython\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
